In [8]:
import pandas as pd

dfproduct3 = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\newProduct.csv"
)
df = pd.read_csv(
    r"C:\Users\pc\Desktop\AIbazaar\AIBazaar\AI\utils\products_descriptions.csv"
)
df["ProductName"] = dfproduct3["Product Name"].str.strip()

df.to_csv("filterPhone2.csv", index=False)
print(f"Original rows: {len(df)}")

Original rows: 370


In [4]:
import pandas as pd

# Giriş ve çıkış dosyası
input_file = "products.csv"
output_file = "product_specs_en.csv"

# Türkçe -> İngilizce kolon çeviri sözlüğü
column_mapping = {
    "Ekran Boyutu": "Display Size",
    "Ekran Teknolojisi": "Display Technology",
    "Piksel Yoğunluğu": "Pixel Density",
    "Batarya Kapasitesi": "Battery Capacity",
    "Kamera Çözünürlüğü": "Camera Resolution",
    "CPU Üretim Teknolojisi": "CPU Manufacturing",
    "İşletim Sistemi": "Operating System",
    "RAM Kapasitesi": "RAM",
    "Dahili Hafıza": "Internal Storage",
    "Hızlı Şarj Desteği": "Fast Charging",
    "Ekran Yenileme Hızı": "Refresh Rate",
    "Model": "Model"
}

# CSV dosyasını oku
df = pd.read_csv("LSTMProducts.csv")

# Her satırı ayrıştır
expanded_rows = []
for _, row in df.iterrows():
    description_parts = row["Description"].split(";")
    specs = {}
    for part in description_parts:
        if ":" in part:
            key, value = part.split(":", 1)
            key = key.strip()
            value = value.strip()
            english_key = column_mapping.get(key, key)  # eşleşmeyeni orijinal bırak
            specs[english_key] = value
    specs["Product Name"] = row["ProductName"]
    expanded_rows.append(specs)

# Yeni DataFrame oluştur
expanded_df = pd.DataFrame(expanded_rows)

# CSV olarak kaydet
expanded_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"{output_file} dosyası İngilizce kolon isimleriyle başarıyla oluşturuldu.")


product_specs_en.csv dosyası İngilizce kolon isimleriyle başarıyla oluşturuldu.
